In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import requests
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bidi import algorithm as bidialg
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get('https://www.yad2.co.il/realestate/forsale?propertyGroup=apartments&priceOnly=1')

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 38.1MB/s]


In [11]:
#Creating a DataFrame for the raw data before cleaning.
raw_data = {'City':[],
            'Street':[],
            'Floor':[],
            'NumOfRooms':[],
            'Size(SQM)':[],
            'Type':[],
            'Price(₪)':[]}
raw_df = pd.DataFrame(raw_data)

In [7]:
content = driver.find_elements(By.CLASS_NAME, "feeditem")

In [ ]:
#First loop for iterating through the first 220 pages.
#Second loop to iterating through each apartment ad.
i = 0
j = 0
while(j < 220):
    i = 0
    for element in content:
        city = element.find_element(By.CLASS_NAME,'subtitle')

        street = element.find_element(By.CLASS_NAME,'title')

        floor = element.find_element(By.ID,'data_floor_{}'.format(i))

        NumOfRooms = element.find_element(By.ID,'data_rooms_{}'.format(i))

        size = element.find_element(By.ID,'data_SquareMeter_{}'.format(i))

        type_ = city.text.split(',')[0]

        price = element.find_element(By.ID,'feed_item_{}_price'.format(i))

        apartment = [city.text ,street.text ,floor.text ,NumOfRooms.text ,size.text ,type_ ,price.text]
        raw_df.loc[len(raw_df.index)] = apartment

        i += 1
        
    j += 1
    driver.get('https://www.yad2.co.il/realestate/forsale?propertyGroup=apartments&priceOnly=1&page={}'.format(j))
    element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'feeditem')))
    content = driver.find_elements(By.CLASS_NAME, "feeditem")

In [ ]:
raw_df.to_csv('apartments.csv')